<a href="https://colab.research.google.com/github/Akmazad/Data-Science-Fundamentals-in-R/blob/main/Modules/Module_2_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling with `tidyr`, `dplyr`, and `tibble` #
**Contents:**



---


1.   Reshaping
2.   Subsetting
3.   Selecting
4.   Grouping and Summarising
5.   Creating new variables
6.   Combining multiple datasets
7.   Set operations and binding
8.   Handling Missing values


---




Loading the packages (`tidyverse` has them all)

In [ ]:
# loading required packages
# dplyr, tidyr and other handy packages - all are in now tidyverse
library(tidyverse)

df = tibble::as_tibble(iris)
df %>% head



---


### 1. Reshaping ###
Change the layout of the dataset

---



In [ ]:
# make a data set wide-to-long format (into key-value pair)
df.wide <- tibble(
  time = as.Date("2009-01-01") + 0:2,
  X1 = rnorm(3, 0, 1),
  X2 = rnorm(3, 0, 2),
  X3 = rnorm(3, 0, 4),
  X4 = rnorm(3, 0, 5)
)
dim(df.wide)
df.long = tidyr::pivot_longer(df.wide,  -time, names_to = "stock", values_to = "price",)
df.long
dim(df.long)

# make long-to-wide (`key-value pair` towards `tabular`)
df.long %>% tidyr::pivot_wider(names_from = stock, values_from = price) # `time` column will be first
df.long %>% tidyr::pivot_wider(names_from = time, values_from = price) # `stock` column will be first

In [ ]:
# expand a data set based on all-possible value combinations of a set of columns
df = tibble(
  x1 = c("A","B","B"),
  x2 = c(1,1,2),
  x3 = c(3,4,3)
)
df
df %>% tidyr::expand(x1,x2)

#  completely define all-possible value combination, even if NA is introduced
df %>% tidyr::complete(x1,x2)

In [ ]:
# Collapse cells across several columns into a single column.
table5 %>% View() # in-built dataset
table5 %>%
  unite(century, year, col = "year", sep = "") %>% # in-built dataset
    unite(country, year, col = "Country_year", sep="_")

table3 %>% View() # in-built data
# Separate each cell in a column into several columns
table3 %>%
  tidyr::separate_wider_delim(rate, delim = "/", names = c("cases", "pop"))
# Separate each cell in a column into several rows.
table3 %>%
  tidyr::separate_longer_delim(rate, delim = "/")

In [ ]:
# arrange the dataset based on columns
dplyr::arrange(mtcars, mpg) # in low-to-high
dplyr::arrange(mtcars, desc(mpg)) # in high-to-low

# rename column names
df = tibble(
  x1 = c("A","B","B"),
  x2 = c(1,1,2),
  x3 = c(3,4,3)
)
df %>% dplyr::rename(
  new_col1 = x1,
  new_col2 = x2,
  new_col3 = x3
)



---

### 2. Subsetting (of rows) - using `dplyr` ###


---



In [ ]:
# Extract rows that meet logical criteria
iris %>% dplyr::filter(Sepal.Length > 7)

# Remove duplicate rows
iris %>% dplyr::distinct()

# Randomly select fraction of rows
iris %>% dplyr::sample_frac(0.5, replace = TRUE)

# Randomly select n rows
iris %>% dplyr::sample_n(10, replace = TRUE)

# Select rows by position
iris %>% dplyr::slice(10:15)



---

### 3. Filtering (of columns) - using `dplyr` ###


---



In [ ]:
# Select columns by name
iris %>% dplyr::select(Sepal.Width, Petal.Length, Species)

# select all columns except a group of column
iris %>% dplyr::select(-c(Petal.Width, Petal.Length))



---

### 4. Group-wise Summary - using `dplyr` ###


---



In [ ]:
# First make groups of rows based on distict values in a column,
# Next, Summarise each groups based on aggregation function
iris %>%
  dplyr::group_by(Species) %>%
  dplyr::summarise_all(.funs = mean)



---

### 5. Creating new variables - using `dplyr` and `tibble` ###


---



In [ ]:
# Compute and append one or more new columns
# iris %>%
#   dplyr::mutate(sepal = Sepal.Length + Sepal.Width)

# make rownames from a given column
mtcars
a <- mtcars %>% tibble::rownames_to_column(var = "C")
a
# make a column from the given rownames of dataset
a %>% tibble::column_to_rownames(var = "C")



---

### 6. Combining multiple datasets - using `dplyr` ###


---



In [ ]:
a = tibble(
  x1 = c("A","B","C"),
  x2 = c(1,2,3)
)
b = tibble(
  x1 = c("A","B","D"),
  x3 = c(T,F,F)
)
a
b

# left-join: Join matching rows from b to a
dplyr::left_join(a, b, by = "x1")

# right-join: Join matching rows from a to b
dplyr::right_join(a, b, by = "x1")

# inner-join: Join data and Retain only rows in both sets
dplyr::inner_join(a, b, by = "x1")

# full-join: Join data and Retain all values, all rows from both datasets a and b
dplyr::full_join(a, b, by = "x1")




---

### 7. Set operations and binding - using `dplyr` ###


---



In [ ]:
y = tibble(
  x1 = c("A","B","C"),
  x2 = c(1,2,3)
)
z = tibble(
  x1 = c("B","C","D"),
  x2 = c(2,3,4)
)
# intersect: Rows that appear in both y and z
dplyr::intersect(y, z)

# union: Rows that appear in either or both y and z
dplyr::union(y, z)

# set difference: Rows that appear in y but not z
dplyr::setdiff(y, z)

# binding rows: Append z to y as new rows
dplyr::bind_rows(y, z)

# binding columns: Append z to y as new columns
dplyr::bind_cols(y, z)




---

### 8. Handling missing values - using `tidyr` ###


---



In [ ]:
# drop missing values (NA)
x = tibble(
  x1 = c("A","B","C","D","E"),
  x2 = c(1, NA, NA, 3, NA)
)
x %>% tidyr::drop_na(x2) # drop the whole row

# Replacing with next/previous values (default: down direction)
x %>% tidyr::fill(x2)

# Replacing all NAs with a fixed value
x %>% tidyr::replace_na(list(x2=2))

---
**Excercise:**

The QS world ranking data that you've loaded in the previous Excercise ([link](https://colab.research.google.com/drive/1uQJaJlUahEcG4nv4Cqf_IAt5PbH-odFf#scrollTo=5oQaqRU-Wh9R&line=4&uniqifier=1)), do the following:



*   Filter the universities that are only in Kindom of Saudi Arabia (Hint: `Country Code` == 'SA')
*   Next sort the universities based on their "International Faculty Rank" column in descending order.

---



In [ ]:
#@title Solution
library(data.table)
library(dplyr)

df = fread("https://docs.google.com/spreadsheets/d/e/2PACX-1vRtySA5U09DJktfiQdTP_j50tCI3h64G6zHFxCDJvkpA8VFgRTn6G9zFGDU9Kwv4s0sianfz7YcvYTD/pub?gid=1872926621&single=true&output=csv")

df %>% dplyr::filter(`Country Code` == 'SA') %>%
  dplyr::arrange(desc(`International Faculty Rank`))